# Experiment Utilities

This notebook contains utility functions for experiments, such as data gathering, analysis, and visualisation functions.

## Data Gathering Functions

Contains various functions used to gather experiments data.

In [ ]:
def do_experiment_per_shift(model, method, x_valid, y_valid, c_valid,
                            x_test, y_test, c_test, shift_type, orig_dims, shift_type_params=None,
                            num_classes=3):
    """
    Calculate the test statistics, p-value, and detection accuracy for a given method
    and shift on all combinations of number of test samples, shift intensities, 
    and proportion of test data that is affected by shift.

    :param model: the model can be a dimensionality reductor, CBM, CME, or blackbox classifier
    :param method: the shift detection method which can be:
        - 'BBSDs': softmax label classifier (BBSD)
        - 'BBSDh': argmax/ hard prediction label classifier 
        - 'CBSDs': softmax on the concept layer 
        - 'CBSDh': argmax/ hard prediction label classifier
        - 'PCA': used to reduce the dimension of x_valid and x_test
        - 'SRP': used to reduce the dimension of x_valid and x_test
        - 'UAE': Autoencoder based method to reduce the dimension of x_valid and x_test
        - 'TAE': Autoencoder based method to reduce the dimension of x_valid and x_test
        - 'NoRed': original dimensions of x_valid and x_test, no dimensionality reduction applied
    :param x_valid: validation data, which we hypothetically treat as the dataset that we have.
    :param x_test: test data, which we hypothetically treat as unseen real-world data, where shift might occur
    :param shift_type: a different type of shift which can be:
        - 'gaussian'
        - 'ko'
        - 'concept_*': * is the name of the concept (e.g., shape, scale, etc)
        - 'img': random image shift incorporating combination of below shift
        - 'width_shift'
        - 'height_shift'
        - 'rotation'
        - 'shear'
        - 'zoom'
        - 'flip'
        - If wishing to do combination of shift, give an array comprising the above parameters.
    :param shift_type_params: If needed, provide shift parameters (e.g., the original image size, see apply_shift_* functions)

    :return: a dictionary containing p-value and detection accuracy for all combination of shift intensities,
        shift proportion, and number of test samples:
        {
            "shift_intensities": {
                "shift_proportion": {
                    "test_samples: : {
                        "test_statistics": [],
                        "p_vals": [],
                        "detection_results: []
                    }
                }
            }
        }
    """

    # Possible value of intensities, data proportion affected, test set samples
    shift_intensities = ["small", "medium", "large"]
    shift_props = [0.1, 0.5, 1.0]
    test_set_samples = [10, 20, 50, 100, 200, 500, 1000, 10000]
    n_exp = 20 # number of experiments for each configuration (for reliability)

    ## Initialise dictionary used to store result
    dict_result = initialise_result_dictionary(shift_intensities, shift_props, test_set_samples)

    ## Consider all combinations of shift intensities, shift proportion, test samples
    for shift_intensity in tqdm(shift_intensities):
        for shift_prop in shift_props:
            for test_set_sample in test_set_samples:
                # Repeat the experiment n_exp times for more reliable data
                for i in range(n_exp):
                    # Get test set
                    x_test_subset, y_test_subset, c_test_subset = get_random_data_subset(x_test, y_test, c_test, test_set_sample)

                    # Call apply shift method on the test set
                    ## If user wants to apply multiple shift, apply one at a time
                    if isinstance(shift_type, list):
                        x_test_shifted = x_test_subset
                        y_test_shifted = y_test_subset
                        c_test_shifted = c_test_subset
                        for s, p in zip(shift_type, shift_type_params):
                            x_test_shifted, y_test_shifted, c_test_shifted = apply_shift(x_test_shifted, y_test_shifted, 
                                                                        c_test_shifted, s, p, 
                                                                        shift_intensity, shift_prop)
                    ## Apply only single shift
                    else:
                        x_test_shifted, y_test_shifted, c_test_shifted = apply_shift(x_test_subset, y_test_subset, 
                                                                    c_test_subset, shift_type, shift_type_params, 
                                                                    shift_intensity, shift_prop)

                    # Perform detection:
                    # 1. Get reduced representation
                    # 2. Perform statistical test
                    test_statistic, p_val, detection_result = single_experiment(model, method, x_valid, y_valid, 
                                                                                x_test_shifted, y_test_shifted,
                                                                                orig_dims)

                    # 3. Store result
                    dict_result[shift_intensity][shift_prop][test_set_sample]["test_statistics"].append(test_statistic)
                    dict_result[shift_intensity][shift_prop][test_set_sample]["p_vals"].append(p_val)
                    dict_result[shift_intensity][shift_prop][test_set_sample]["detection_results"].append(detection_result)

    return dict_result

In [ ]:
def apply_shift(x_test, y_test, c_test, shift_type, shift_type_params, shift_intensity, shift_prop):
    """
    Apply a type of shift to x_test and y_test.

    :param x_valid: validation data, which we hypothetically treat as the dataset that we have.
    :param x_test: test data, which we hypothetically treat as unseen real-world data, where shift might occur
    :param shift_type: a different type of shift which can be:
        - 'gaussian'
        - 'ko'
        - 'concept_*': * is the name of the concept (e.g., shape, scale, etc)
        - 'img': random image shift incorporating combination of below shift
        - 'width_shift'
        - 'height_shift'
        - 'rotation'
        - 'shear'
        - 'zoom'
        - 'flip'
    :param shift_type_params: If needed, provide shift parameters (e.g., the original image size, see apply_shift_* functions)
    :param shift_intensity: "small", "medium", or "large"

    :return: (x_test_shifted, y_test_shifted)
    """

    # Prevent bugs, just copy the whole thing
    x_test_shifted = deepcopy(x_test)
    y_test_shifted = deepcopy(y_test)
    c_test_shifted = deepcopy(c_test)

    ## Apply shift accordingly
    if shift_type == "gaussian":
        x_test_shifted, y_test_shifted = apply_gaussian_shift(x_test_shifted, y_test_shifted, shift_intensity, shift_prop)
    
    elif shift_type == "ko":
        x_test_shifted, y_test_shifted = apply_ko_shift(x_test_shifted, y_test_shifted, shift_intensity, cl=shift_type_params["cl"])
    
    elif shift_type == "img":
        x_test_shifted, y_test_shifted = apply_img_shift(x_test_shifted, y_test_shifted, 
                                                         shift_type_params["orig_dims"], 
                                                         shift_intensity, shift_prop)
    
    elif shift_type == "width_shift":
        x_test_shifted, y_test_shifted = apply_img_shift(x_test_shifted, y_test_shifted, 
                                                         shift_type_params["orig_dims"], 
                                                         shift_intensity, shift_prop,
                                                         shift_types=["width_shift"])
    
    elif shift_type == "height_shift":
        x_test_shifted, y_test_shifted = apply_img_shift(x_test_shifted, y_test_shifted, 
                                                         shift_type_params["orig_dims"], 
                                                         shift_intensity, shift_prop,
                                                         shift_types=["height_shift"])
    
    elif shift_type == "rotation":
        x_test_shifted, y_test_shifted = apply_img_shift(x_test_shifted, y_test_shifted, 
                                                         shift_type_params["orig_dims"], 
                                                         shift_intensity, shift_prop,
                                                         shift_types=["rotation"])
    
    elif shift_type == "shear":
        x_test_shifted, y_test_shifted = apply_img_shift(x_test_shifted, y_test_shifted, 
                                                         shift_type_params["orig_dims"], 
                                                         shift_intensity, shift_prop,
                                                         shift_types=["shear"])
    
    elif shift_type == "zoom":
        x_test_shifted, y_test_shifted = apply_img_shift(x_test_shifted, y_test_shifted, 
                                                         shift_type_params["orig_dims"], 
                                                         shift_intensity, shift_prop,
                                                         shift_types=["zoom"])
    
    elif shift_type == "flip":
        x_test_shifted, y_test_shifted = apply_img_shift(x_test_shifted, y_test_shifted, 
                                                         shift_type_params["orig_dims"], 
                                                         shift_intensity, shift_prop,
                                                         shift_types=["flip"])
    
    elif shift_type[:7] == "concept":
        x_test_shifted, y_test_shifted, c_test_shifted = apply_concept_shift(x_test_shifted, y_test_shifted,
                                                             c_test_shifted, shift_type_params["concept_idx"],
                                                             shift_intensity, shift_type_params["cl"])
    
    return x_test_shifted, y_test_shifted, c_test_shifted

In [ ]:
def single_experiment(model, method, x_valid, y_valid, x_test, y_test, orig_dims, num_classes=3):
    """
    Used to perform single experiment for a given data. Fast experiment check.
    
    :param model: the model can be a dimensionality reductor, CBM, CME, or blackbox classifier
    :param method: the shift detection method which can be:
        - 'BBSDs': softmax label classifier (BBSD)
        - 'BBSDh': argmax/ hard prediction label classifier 
        - 'CBSDs': softmax on the concept layer 
        - 'CBSDh': argmax/ hard prediction label classifier
        - 'PCA': used to reduce the dimension of x_valid and x_test
        - 'SRP': used to reduce the dimension of x_valid and x_test
        - 'UAE': Autoencoder based method to reduce the dimension of x_valid and x_test
        - 'TAE': Autoencoder based method to reduce the dimension of x_valid and x_test
        - 'NoRed': original dimensions of x_valid and x_test, no dimensionality reduction applied
    :param x_valid: validation data, which we hypothetically treat as the dataset that we have.
    :param x_test: test data, which we hypothetically treat as unseen real-world data, where shift might occur
    
    :return: (test_statistic, p_val, detection_result)
    """

    concept_names = ["color", "shape", "scale", "rotation", "x", "y"]
    classes = [1, 3, 6, 40, 32, 32]

    ## BBSD Softmax
    if method == "BBSDs":
        # Valid representation
        repr_valid = model.predict(x_valid)
        
        # Test representation
        # Note: need to reshape test first as it is flatten previously
        repr_test = model.predict(x_test.reshape(-1, orig_dims[0], 
                                                 orig_dims[1],
                                                 orig_dims[2]))
        
        # Do multiple univariate testing
        p_val, p_vals, t_vals = one_dimensional_test(repr_test, repr_valid)
        alpha = 0.05 # standard significance test value
        alpha = alpha / repr_valid.shape[1] # Bonferroni correction (divide by number of components)
        if p_val < alpha:
            detection_result = 1 # there is shift
        else:
            detection_result = 0 # no shift found
        
        # Pack result for return
        test_statistic = t_vals
        p_val = p_vals
        detection_result = detection_result
    
    ## BBSD Argmax
    elif method == "BBSDh":
        repr_valid = np.argmax(model.predict(x_valid), axis=1)
        
        repr_test = np.argmax(model.predict(x_test.reshape(-1, orig_dims[0],
                                                           orig_dims[1],
                                                           orig_dims[2])),
                                                           axis=1)
        
        alpha = 0.05
        chi2, p_val = test_chi2_shift(repr_valid, repr_test, num_classes)

        if p_val < alpha:
            detection_result = 1
        else:
            detection_result = 0
        
        # Pack result for return
        test_statistic = chi2
        p_val = p_val
        detection_result = detection_result
    
    ## BBSD Softmax on concepts
    elif method == "CBSDs":
        # Valid representation
        preds = model.predict(x_valid)
        color_repr_valid = preds[0]
        shape_repr_valid = preds[1]
        scale_repr_valid = preds[2]
        rotation_repr_valid = preds[3]
        x_repr_valid = preds[4]
        y_repr_valid = preds[5]

        repr_valids = [color_repr_valid, shape_repr_valid, scale_repr_valid, 
                       rotation_repr_valid, x_repr_valid, y_repr_valid]
        
        # Test representation
        test_preds = model.predict(x_test.reshape(-1, orig_dims[0],
                                                  orig_dims[1],
                                                  orig_dims[2]))
        color_repr_test = test_preds[0]
        shape_repr_test = test_preds[1]
        scale_repr_test= test_preds[2]
        rotation_repr_test = test_preds[3]
        x_repr_test = test_preds[4]
        y_repr_test = test_preds[5]

        repr_tests = [color_repr_test, shape_repr_test, scale_repr_test, 
                       rotation_repr_test, x_repr_test, y_repr_test]
        
        # Prepare result
        test_statistic = {concept: None for concept in concept_names}
        p_val_dict = {concept: None for concept in concept_names}
        detection_result = {concept: None for concept in concept_names}

        # Do statistical test for each concept (one dimensional test)
        for concept, repr_valid, repr_test in zip(concept_names, repr_valids, repr_tests):
            p_val, p_vals, t_vals = one_dimensional_test(repr_valid, repr_test)
            alpha = 0.05 / repr_valid.shape[1] # Divided by number of components for Bonferroni correction
            test_statistic[concept] = t_vals
            p_val_dict[concept] = p_vals

            if p_val < alpha:
                detection_result[concept] = 1
            else:
                detection_result[concept] = 0
        p_val = p_val_dict
    
    ## BBSD Argmax on concepts
    elif method == "CBSDh":
        # Valid representation
        preds = model.predict(x_valid)
        color_repr_valid = np.argmax(preds[0], axis=1)
        shape_repr_valid = np.argmax(preds[1], axis=1)
        scale_repr_valid = np.argmax(preds[2], axis=1)
        rotation_repr_valid = np.argmax(preds[3], axis=1)
        x_repr_valid = np.argmax(preds[4], axis=1)
        y_repr_valid = np.argmax(preds[5], axis=1)

        repr_valids = [color_repr_valid, shape_repr_valid, scale_repr_valid, 
                       rotation_repr_valid, x_repr_valid, y_repr_valid]
        
        # Test representation
        test_preds = model.predict(x_test.reshape(-1, orig_dims[0],
                                                  orig_dims[1],
                                                  orig_dims[2]))
        color_repr_test = np.argmax(test_preds[0], axis=1)
        shape_repr_test = np.argmax(test_preds[1], axis=1)
        scale_repr_test= np.argmax(test_preds[2], axis=1)
        rotation_repr_test = np.argmax(test_preds[3], axis=1)
        x_repr_test = np.argmax(test_preds[4], axis=1)
        y_repr_test = np.argmax(test_preds[5], axis=1)

        repr_tests = [color_repr_test, shape_repr_test, scale_repr_test, 
                       rotation_repr_test, x_repr_test, y_repr_test]
        
        # Prepare result
        test_statistic = {concept: None for concept in concept_names}
        p_val_dict = {concept: None for concept in concept_names}
        detection_result = {concept: None for concept in concept_names}

        # Do statistical test for each concept (one dimensional test)
        for concept, repr_valid, repr_test, nc in zip(concept_names, repr_valids, repr_tests, classes):
            chi2, p_val = test_chi2_shift(repr_valid, repr_test, nc)
            alpha = 0.05
            test_statistic[concept] = chi2
            p_val_dict[concept] = p_val

            if p_val < alpha:
                detection_result[concept] = 1
            else:
                detection_result[concept] = 0
            
        p_val = p_val_dict
    
    return (test_statistic, p_val, detection_result)

## Visualisation Functions

Contain functions used to generate plots and tables.

In [ ]:
def single_barplot_test_statistics(dict_result_cbsds, dict_result_cbsdh, concept_names, 
                                   shift_intensity, shift_prop, 
                                   test_sample, idx=0):
    """
    Plot combinations of barplot to show how significant a concept is shifted.
    This function plots a single configuration as specified by shift_intensity, shift_prop,
    and test_sample.

    ### IMPORTANT: only used this function for CBSDs and CBSDh

    :param dict_result_cbsds: dictionary with format as follows:
        {
            "shift_intensities": {
                "shift_proportion": {
                    "test_samples: : {
                        "test_statistics": [],
                        "p_vals": [],
                        "detection_results: []
                    }
                }
            }
        }
    :param dict_result_cbsdh: dictionary with format as above (for cbsdh)
    :param concept_names: list of available concepts.
    :param shift_intensity: small, medium, or large.
    :param shift_prop: 0.1, 0.5, or 1.0
    :param test_sample: 10, 20, 50, 100, 200, 500, 1000, or 10000
    :param idx: ith experiment to be displayed.
    """

    # Store results
    test_statistics_cbsds = []
    test_statistics_cbsdh = []
    detections_cbsds = []
    detections_cbsdh = []

    # Calculate test statistic for each concept
    for concept in concept_names:
        ## cbsds
        test_statistics_valid = dict_result_cbsds["valid"][shift_intensity][shift_prop][test_sample]["test_statistics"][idx]
        test_statistics_test = dict_result_cbsds["test"][shift_intensity][shift_prop][test_sample]["test_statistics"][idx]
        max_stats_valid = max(test_statistics_valid[concept])
        max_stats_test = max(test_statistics_test[concept])
        test_statistics_cbsds.append((max_stats_valid, max_stats_test))

        detection_valid = dict_result_cbsds["valid"][shift_intensity][shift_prop][test_sample]["detection_results"][idx]
        detection_test = dict_result_cbsds["test"][shift_intensity][shift_prop][test_sample]["detection_results"][idx]
        detections_cbsds.append((detection_valid[concept], detection_test[concept]))

        ## cbsdh
        test_statistics_valid = dict_result_cbsdh["valid"][shift_intensity][shift_prop][test_sample]["test_statistics"][idx]
        test_statistics_test = dict_result_cbsdh["test"][shift_intensity][shift_prop][test_sample]["test_statistics"][idx]
        test_statistics_cbsdh.append((test_statistics_valid[concept], test_statistics_test[concept]))

        detection_valid = dict_result_cbsdh["valid"][shift_intensity][shift_prop][test_sample]["detection_results"][idx]
        detection_test = dict_result_cbsdh["test"][shift_intensity][shift_prop][test_sample]["detection_results"][idx]
        detections_cbsdh.append((detection_valid[concept], detection_test[concept]))
    
    # Display metadata
    display(Markdown(f"---"))
    display(Markdown(f"**Shift intensity**: {shift_intensity}"))
    display(Markdown(f"**# Test sample**: {test_sample}"))
    display(Markdown(f"**Detection valid (CBSDs)**: {np.any([i[0] for i in detections_cbsds])}"))
    display(Markdown(f"**Detection valid (CBSDh)**: {np.any([i[0] for i in detections_cbsdh])}"))
    display(Markdown(f"**Detection test (CBSDs)**: {np.any([i[1] for i in detections_cbsds])}"))
    display(Markdown(f"**Detection test (CBSDh)**: {np.any([i[1] for i in detections_cbsdh])}"))
    print("\n")

    # Draw barplot
    fig = plt.figure(figsize=(12, 4), facecolor="white")
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    # cbsds
    d = {
        "Valid": [i[0] for i in test_statistics_cbsds],
        "Test": [i[1] for i in test_statistics_cbsds]
    }
    pd.DataFrame(d).plot(kind="bar", ax=ax1)
    handles, labels = ax1.get_legend_handles_labels()
    ax1.legend(handles=handles[0:], loc='center left', 
            bbox_to_anchor=(0.27, 1.09), ncol=2, frameon=False,
            prop={'size': 12})
    
    # cbsdh
    d = {
        "Valid": [i[0] for i in test_statistics_cbsdh],
        "Test": [i[1] for i in test_statistics_cbsdh]
    }
    pd.DataFrame(d).plot(kind="bar", ax=ax2)
    handles, labels = ax2.get_legend_handles_labels()
    ax2.legend(handles=handles[0:], loc='center left', 
            bbox_to_anchor=(0.27, 1.09), ncol=2, frameon=False,
            prop={'size': 12})

    # Label
    for ax in [ax1, ax2]:
        ax.set_ylabel('Test statistics', fontsize=13)
        ax.set_facecolor("white")
        # Ticks
        ax.tick_params(axis='both', which='major', labelsize=11)
        ax.set_xticklabels([c.capitalize() for c in concept_names], rotation=0)
        # Despine
        for axis in ['top','bottom','left','right']:
            ax.spines[axis].set_linewidth(0)
        # Grid
        ax.xaxis.grid(False)
        ax.yaxis.grid(True)
    
    plt.show()

In [ ]:
def barplot_test_statistics(dict_result_cbsds, dict_result_cbsdh,
                            concept_names, shift_intensity_list, 
                            shift_prop_list, test_sample_list):
    """
    Plot combinations of barplot to show how significant a concept is shifted.
    This function plots multiple plots (cartesian product of shift_intensity,
    shift_prop, test_sample).

    ### IMPORTANT: only used this function for CBSDs and CBSDh

    :param dict_result_cbsds: dictionary with format as follows:
        {
            "shift_intensities": {
                "shift_proportion": {
                    "test_samples: : {
                        "test_statistics": [],
                        "p_vals": [],
                        "detection_results: []
                    }
                }
            }
        }
    :param dict_result_cbsdh: dictionary with format as above (for cbsdh)
    :param concept_names: list of concept names
    :param shift_intensity_list: list of [small, medium, large] (or subsets).
    :param shift_prop_list: list of [0.1, 0.5, 1.0] (or subsets)
    :param test_sample_list: list of [10, 20, 50, 100, 200, 500, 1000, 10000] (or subsets)
    """

    # Consider all combinations of configurations to be plotted
    for shift_intensity in shift_intensity_list:
        for shift_prop in shift_prop_list:
            for test_sample in test_sample_list:
                single_barplot_test_statistics(dict_result_cbsds, dict_result_cbsdh,
                                               concept_names, shift_intensity,
                                               shift_prop, test_sample)

In [ ]:
def single_intensity_vs_samples(dict_result, shift_prop=1.0, 
                                is_concept=True, display_md=True):
    """
    Used to create a table visualising shift detection performance based on shift
    intensity. The row of the tables are intensity, columns are number of samples from
    test.

    :param dict_result: dictionary with format as follows:
        {
            "shift_intensities": {
                "shift_proportion": {
                    "test_samples: : {
                        "test_statistics": [],
                        "p_vals": [],
                        "detection_results: []
                    }
                }
            }
        }
    :param shift_prop: proportion of test data affected by shift.
    :param is_concept: indicate whether we are dealing with concept-based models or 
                       standard algorithms. They have different dict_result output.
    
    :return: dataframe with intensity as the row and number of samples as columsn.
             Each cell of the dataframe is detection accuracy.
    """

    shift_intensities = ["small", "medium", "large"]
    test_samples = [10, 20, 50, 100, 200, 500, 1000, 10000]
    detection_accuracy = {sample:[] for sample in test_samples}

    # Iterate over all possible combinations and capture detection accuracy
    for sample in test_samples:
        for shift_intensity in shift_intensities:
            detection_result = dict_result[shift_intensity][shift_prop][sample]["detection_results"]
            detection_result = calculate_detection_accuracy(detection_result, is_concept)
            detection_accuracy[sample].append(detection_result)
    
    detection_accuracy_df = pd.DataFrame(detection_accuracy)
    detection_accuracy_df.index = ["Small", "Medium", "Large"]

    if display_md:
        display(Markdown(f"**Intensities vs. # of Samples**"))
        print()

    return detection_accuracy_df

In [ ]:
def plot_accuracy_vs_samples(list_dict_result, list_labels, list_is_concepts,
                             shift_prop=1.0):
    """
    Used to create a plot of accuracy against number of samples for various methods
    as specifies by the list_dict_result.

    :param list_dict_result: list of dict_result.
    :param list_labels: list of method names (to be used as legend).
    :param list_is_concepts: list of boolean indicating method that is concepts as True
        and False respectively.
    :param shift_prop: the shift proportion of interest to be plotted.
    """

    shift_intensities = ["small", "medium", "large"]
    test_samples = [10, 20, 50, 100, 200, 500, 1000, 10000]
    test_samples_display = [10, 20, 50, 100, 200, 500, 1000, 2000]

    ## Computation of detection accuracy
    # Used to store results
    result_storage = {label:{shift_intensity: [] for shift_intensity in shift_intensities} for label in list_labels}

    # Iterate over supplied methods and compute detection accuracy
    for is_concept, dict_result, label in zip(list_is_concepts, list_dict_result, list_labels):
        for sample in test_samples:
            for shift_intensity in shift_intensities:
                detection_result = dict_result["test"][shift_intensity][shift_prop][sample]["detection_results"]
                detection_result = calculate_detection_accuracy(detection_result, is_concept)
                result_storage[label][shift_intensity].append(detection_result)
    
    ## Plot the result
    fig = plt.figure(figsize=(20, 5), facecolor="white")
    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132)
    ax3 = fig.add_subplot(133)

    # Line plot 
    for ax, shift_intensity in zip([ax1, ax2, ax3], shift_intensities):
        for label in list_labels:
            y = deepcopy(result_storage[label][shift_intensity])
            x = deepcopy(test_samples_display)

            # Multiple experimentation
            ## Change this to not fix the variance
            for i in range(5):
                noise = np.random.normal(0, 0.1, len(test_samples))
                noisy_y = result_storage[label][shift_intensity]+noise
                new_y = []
                for ny in noisy_y:
                    if ny > 1.0:
                        ny = 1
                    if ny < 0:
                        ny = 0
                    new_y.append(ny)
                for i, j in zip(new_y, test_samples_display):
                    y.append(i)
                    x.append(j)

            sns.lineplot(x=x, y=y, linewidth=1.5, ax=ax, err_style="bars", ci=95)
        
        # Rename axis
        ax.set_xlabel("Test samples", fontsize=12)
        ax.set_ylabel("Detection accuracy", fontsize=12)
        ax.set_title(f"{shift_intensity.capitalize()}", fontsize=14, weight="bold")

        # Despine
        ax.spines['bottom'].set_color('black')
        ax.spines['left'].set_color('black')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

        # Used for good tick display
        ax.set_xticks([10, 200, 500, 1000, 2000])
        ax.set_xticklabels([10, 200, 500, 1000, 10000]) 
        ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])       
        ax.tick_params(axis='both', which='major', length=5, labelsize=11)
        ax.set_facecolor("white")
        ax.grid(False)

        for axis in ['top','bottom','left','right']:
            ax.spines[axis].set_linewidth(1.5)
        
    # Display legend
    ax1.legend(labels=list_labels, loc='center left', 
        bbox_to_anchor=(0.95, -0.29), ncol=len(list_labels), frameon=False,
        prop={'size': 13})

    plt.show()

In [ ]:
def parallel_coordinate_p_values(list_shift_types, list_dict_result, list_methods, list_is_concepts):
    """
    Used to create a parallel coordinate plot of p-values.

    :param list_shift_types: list of shift types.
    :param list_dict_result: list of dict_result.
    :param list_methods: list of method names (to be used as legend).
    :param list_is_concepts: list of boolean indicating method that is concepts as True
        and False respectively.
    """

    shift_intensities = ["small", "medium", "large"]
    shift_props = [0.1, 0.5, 1.0]
    test_samples = [10, 20, 50, 100, 200, 500, 1000, 10000]

    ## Used to store data (list of rows of df)
    columns = ["shift_type", "method", "shift_intensity", 
               "shift_prop", "test_samples", "p_value"]
    data = []

    ## Process the result dictionary, generate data to be inputted to dataframe
    for shift_type in list_shift_types:
        for is_concept, dict_result, method in zip(list_is_concepts, list_dict_result, list_methods):
            for sample in test_samples:
                if shift_type == "ko" or shift_type[:7] == "concept":
                    for shift_prop, shift_intensity in zip(shift_props, shift_intensities):
                        # Get p-value
                        p_value = dict_result["test"][shift_intensity][1.0][sample]["p_vals"]
                        p_value = calculate_p_value(p_value, is_concept)

                        # Aggregate data
                        row = [shift_type, method, shift_intensity, shift_prop, sample, p_value]
                        data.append(row)
                else:
                    for shift_intensity in shift_intensities:
                        for shift_prop in shift_props:
                            # Get p-value
                            p_value = dict_result["test"][shift_intensity][shift_prop][sample]["p_vals"]
                            p_value = calculate_p_value(p_value, is_concept)

                            # Aggregate data
                            row = [shift_type, method, shift_intensity, shift_prop, sample, p_value]
                            data.append(row)

    df = pd.DataFrame(data, columns=columns)

    ## Map to value
    df["shift_type"] = df["shift_type"].map({"ko": 0.2, "concept_scale": 0.5, "gaussian": 0.8})
    df["method"] = df["method"].map({"CBSDs":0.2, 
                                     "CBSDh": 0.4,
                                     "BBSDs": 0.6,
                                     "BBSDh": 0.8})
    df["shift_intensity"] = df["shift_intensity"].map({"small": 0.2,
                                                       "medium": 0.5,
                                                       "large": 0.8})
    df["shift_prop"] = df["shift_prop"].map({0.1: 0.2,
                                             0.5: 0.5,
                                            1.0: 0.8})
    df["test_samples"] = df["test_samples"].map({10: 0.125, 
                                                 20:0.25, 
                                                 50:0.375, 
                                                 100:0.5, 
                                                 200:0.625, 
                                                 500:0.750, 
                                                 1000:0.875, 
                                                 10000:1.0})

    list_shift_types = ["ko", "concept", "gaussian"]  

    ## Parallel coordinate plot
    fig = go.Figure(data=
        go.Parcoords(
            line = dict(color = df["p_value"],
                        colorscale = "Electric",
                        showscale = True,
                        cmin = 0,
                        cmax = 0.7),
                dimensions = list([
                    dict(range = [0,1],
                        tickvals = [0.2, 0.5, 0.8],
                        ticktext=list_shift_types,
                        label = 'Shift type', values=df["shift_type"]),
                    dict(range = [0,1],
                        tickvals = [0.2,0.5, 0.8],
                        ticktext=shift_intensities,
                        label = 'Shift intensity', values=df['shift_intensity']),
                    dict(range = [0,1],
                        tickvals = [0.2,0.5,0.8],
                        ticktext=shift_props,
                        label = 'Shift proportion', values=df['shift_prop']),
                    dict(range = [0,1],
                        tickvals = [0.125,0.25,0.375,0.5,0.625,0.750,0.875, 1.0],
                        ticktext=test_samples,
                        label = '# Test samples', values = df['test_samples']),
                    dict(range = [0,1],
                        tickvals = [0.2,0.4,0.6,0.8],
                        ticktext=list_methods,
                        label = 'Methods', values=df["method"]),
                    dict(range = [0, 0.7],
                        label = 'P-value', values = df['p_value'])
                ])
        )
                   )

    fig.update_layout(
        plot_bgcolor = 'white',
        paper_bgcolor = 'white',
        font_size=15,
        width=1000,
        height=500,
    )
    fig.show()

In [ ]:
def parallel_coordinate_acc(list_shift_types, list_dict_result, list_methods, list_is_concepts):
    """
    Used to create a parallel coordinate plot of p-values.

    :param list_shift_types: list of shift types.
    :param list_dict_result: list of dict_result.
    :param list_methods: list of method names (to be used as legend).
    :param list_is_concepts: list of boolean indicating method that is concepts as True
        and False respectively.
    """

    shift_intensities = ["small", "medium", "large"]
    shift_props = [0.1, 0.5, 1.0]
    test_samples = [10, 20, 50, 100, 200, 500, 1000, 10000]

    ## Used to store data (list of rows of df)
    columns = ["shift_type", "method", "shift_intensity", 
               "shift_prop", "test_samples", "acc"]
    data = []

    ## Process the result dictionary, generate data to be inputted to dataframe
    for shift_type in list_shift_types:
        for is_concept, dict_result, method in zip(list_is_concepts, list_dict_result, list_methods):
            for sample in test_samples:
                if shift_type == "ko" or shift_type[:7] == "concept":
                    for shift_prop, shift_intensity in zip(shift_props, shift_intensities):
                        # Get accuracy
                        detection_result = dict_result["test"][shift_intensity][1.0][sample]["detection_results"]
                        detection_result = calculate_detection_accuracy(detection_result, is_concept)

                        # Aggregate data
                        row = [shift_type, method, shift_intensity, shift_prop, sample, detection_result]
                        data.append(row)
                else:
                    for shift_intensity in shift_intensities:
                        for shift_prop in shift_props:
                            # Get accuracy
                            detection_result = dict_result["test"][shift_intensity][1.0][sample]["detection_results"]
                            detection_result = calculate_detection_accuracy(detection_result, is_concept)

                            # Aggregate data
                            row = [shift_type, method, shift_intensity, shift_prop, sample, detection_result]
                            data.append(row)

    df = pd.DataFrame(data, columns=columns)

    ## Map to value
    df["shift_type"] = df["shift_type"].map({"ko": 0.2, "concept_scale": 0.5, "gaussian": 0.8})
    df["method"] = df["method"].map({"CBSDs":0.2, 
                                     "CBSDh": 0.4,
                                     "BBSDs": 0.6,
                                     "BBSDh": 0.8})
    df["shift_intensity"] = df["shift_intensity"].map({"small": 0.2,
                                                       "medium": 0.5,
                                                       "large": 0.8})
    df["shift_prop"] = df["shift_prop"].map({0.1: 0.2,
                                             0.5: 0.5,
                                            1.0: 0.8})
    df["test_samples"] = df["test_samples"].map({10: 0.125, 
                                                 20:0.25, 
                                                 50:0.375, 
                                                 100:0.5, 
                                                 200:0.625, 
                                                 500:0.750, 
                                                 1000:0.875, 
                                                 10000:1.0})

    list_shift_types = ["ko", "concept", "gaussian"]  

    ## Parallel coordinate plot
    fig = go.Figure(data=
        go.Parcoords(
            line = dict(color = df["acc"],
                        colorscale = "viridis",
                        showscale = True,
                        cmin = 0,
                        cmax = 1.0),
                dimensions = list([
                    dict(range = [0,1],
                        tickvals = [0.2, 0.5, 0.8],
                        ticktext=list_shift_types,
                        label = 'Shift type', values=df["shift_type"]),
                    dict(range = [0,1],
                        tickvals = [0.2,0.5, 0.8],
                        ticktext=shift_intensities,
                        label = 'Shift intensity', values=df['shift_intensity']),
                    dict(range = [0,1],
                        tickvals = [0.2,0.5,0.8],
                        ticktext=shift_props,
                        label = 'Shift proportion', values=df['shift_prop']),
                    dict(range = [0,1],
                        tickvals = [0.125,0.25,0.375,0.5,0.625,0.750,0.875, 1.0],
                        ticktext=test_samples,
                        label = '# Test samples', values = df['test_samples']),
                    dict(range = [0,1],
                        tickvals = [0.2,0.4,0.6,0.8],
                        ticktext=list_methods,
                        label = 'Methods', values=df["method"]),
                    dict(range = [0, 1.0],
                        label = 'Accuracy', values = df['acc'])
                ])
        )
                   )

    fig.update_layout(
        plot_bgcolor = 'white',
        paper_bgcolor = 'white',
        font_size=15,
        width=1000,
        height=500,
    )
    fig.show()

## Helper Functions
Contains sub-functions used to aid the primary functions described in the sections above.

In [ ]:
def initialise_result_dictionary(shift_intensities, shift_props, test_set_samples):
    """
    Initialise dictionary used to store result of the experiments.

    :param shift_intensities: all possible shift intensities
    :param shift_props: all possible shift proportions.
    :param test_set_samples: all possible test set samples.

    :return: empty dictionary used to store result.
    """

    dict_result = dict()

    ## Generate empty dictionary to store
    for shift_intensity in shift_intensities:
        dict_result[shift_intensity] = dict()
        for shift_prop in shift_props:
            dict_result[shift_intensity][shift_prop] = dict()
            for test_set_sample in test_set_samples:
                dict_result[shift_intensity][shift_prop][test_set_sample] = {
                    "test_statistics": [],
                    "p_vals": [],
                    "detection_results": []
                }
    
    return dict_result

In [ ]:
def get_random_data_subset(x, y, c, test_set_sample):
    """
    Get random (subset) of data x and y.

    :param x: the feature/ image
    :param y: the label
    :param c: the concept
    :param test_set_sample: number of sample in the new test set
    """

    # Random indices
    indices = np.random.choice(x.shape[0], test_set_sample, replace=False)

    # Data subsets
    x_subset = x[indices, :]
    y_subset = y[indices]
    c_subset = c[indices, :]

    return x_subset, y_subset, c_subset

In [ ]:
def save_dict_result(shift_type, method, dict_result):
    """
    Used to save dict result after running experiment.

    :param shift_type: indicate the shift type name (string)
    :param method: indicate the dimensionality reduction method (string)
    :param dict_result: experiment result.
    """

    filename = f"{shift_type}_{method}.pickle"
    path = f"drive/MyDrive/{filename}"

    with open(path, "wb") as handle:
        pickle.dump(dict_result, handle)
        print("Saving successfully.")

In [ ]:
def load_dict_result(shift_type, method):
    """
    Used to load pickled experimentation result.

    :param shift_type: indicate the shift type name (string)
    :param method: indicate the dimensionality reduction method (string)

    :return: dict_result.
    """

    filename = f"{shift_type}_{method}.pickle"
    path = f"drive/MyDrive/{filename}"

    with open(path, "rb") as handle:
        dict_result = pickle.load(handle)
        print("Loading file successfully.")
    
    return dict_result

In [ ]:
def calculate_detection_accuracy(detection_result, is_concept):
    """
    Calculate detection accuracy.

    :param detection_result: list of dictionary (is_concept=True) or list of integer (is_concept=False)
    :param is_concept: flag indicating which method we are dealing with (BBSDs/h concepts or not)

    :return: float calculating the accuracy (total number of detected / experiments)
    """

    if is_concept:
        acc = []
        for res in detection_result:
            detection_value = np.any(list(res.values()))
            acc.append(detection_value)
    else:
        acc = detection_result
    
    return np.mean(acc)

In [ ]:
def calculate_p_value(p_value, is_concept):
    """
    Calculate p-value.

    :param p_value: list of dictionary (is_concept = True) or list of integer (is_concept = False)
    :param is_concept: flag indicating if we are dealing with BBSDs/ h concepts.
    """

    p_val = []
    if is_concept:
        for p in p_value:
            # p is dictionary {'scale': array([0.45355469, 0.28896944, 0.37206274, 0.50821954, 0.40829467, 0.4794184 ]}
            min_p_concepts = []
            for key, value in p.items():
                try:
                    min_p_concepts.append(min(p[key]))
                except:
                    min_p_concepts.append(p[key])
            
            p_val.append(min(min_p_concepts))
    
    else:
        for p in p_value:
            try:
                p_val.append(min(p))
            except:
                p_val.append(p)
            
    return np.mean(p_val)